# J1044 ABUNDANCE SCALES AND METALLICITIES

## Importing Packages

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from astropy.io import fits as pyfits
from astropy.table import Table
from astropy.io import ascii

In [2]:
import os
import sys
import re

## Metallicity Determination by Hand

In [3]:
logOH12_gal = 7.47
logOH_gal = logOH12_gal - 12

logOH_gal

-4.53

In [4]:
logOH_solar = -3.31

print("J1044 O/H:", logOH_gal)
print("Solar O/H:", logOH_solar)

J1044 O/H: -4.53
Solar O/H: -3.31


To find the metals and grains, we have to find the value of J1044 O/H over solar O/H. However, note that this is a log value and we want to make it linear.

In [5]:
metallicity = (10**logOH_gal) / (10**logOH_solar)

print("J1044 metals and grains:", "{:.2f}".format(metallicity))

J1044 metals and grains: 0.06


## Defining a Metallicity Scale Function

In [6]:
def find_metals(galaxy_OH12):
    solar_logOH = -3.31
    galaxy_logOH = galaxy_OH12 - 12

    galaxy_metals = (10**galaxy_logOH) / (10**solar_logOH)

    return float("{:.2f}".format(galaxy_metals))

In [7]:
def find_metals_with_err(galaxy_OH12, galaxy_OH12_err):
    solar_logOH = -3.31
    galaxy_logOH = galaxy_OH12 - 12
    galaxy_logOH_err = (galaxy_OH12 + galaxy_OH12_err) - 12

    galaxy_metals = (10**galaxy_logOH) / (10**solar_logOH)
    galaxy_metals_1sig = (10**galaxy_logOH_err) / (10**solar_logOH)

    # return float("{.2f,.2f}".format(galaxy_metals,galaxy_metals_1sig))
    return galaxy_metals, galaxy_metals_1sig

In [8]:
if find_metals(7.47) == float("{:.2f}".format(metallicity)):
    print("Metallicity function works!")
    print("Our metals and grains value for J1044 is:", find_metals(7.47))
else:
    print("Not quite right there.")

Metallicity function works!
Our metals and grains value for J1044 is: 0.06


In [11]:
find_metals(7.44 - 0.3)

0.03

In [12]:
find_metals(7.44 + 0.3)

0.11

# ELEMENTAL SCALE FUNCTIONS

## Defining a Nitrogen Scale Function

In [13]:
def N_scale(galaxy_logNO):
    solar_logNH = -4.17
    solar_logOH = -3.31
    solar_NO = (10**solar_logNH) * (1 / (10**solar_logOH))

    scale = (10**galaxy_logNO) / (solar_NO)

    return float("{:.2f}".format(scale))

In [17]:
N_scale(-1.35 - 0.06)

0.28

In [15]:
N_scale(-1.35 + 0.06)

0.37

## Defining a Carbon Scale Function

In [18]:
def C_scale(galaxy_logCO):
    solar_logCH = -3.57
    solar_logOH = -3.31
    solar_CO = (10**solar_logCH) * (1 / (10**solar_logOH))

    scale = (10**galaxy_logCO) / (solar_CO)

    return float("{:.2f}".format(scale))

In [19]:
C_scale(-0.73 - 0.09)

0.28

In [20]:
C_scale(-0.73 + 0.09)

0.42

## Defining a Silicon Scale Function

In [46]:
def Si_scale(galaxy_logSiO):
    solar_logSiH = -4.49
    solar_logOH = -3.31
    solar_SiO = (10**solar_logSiH) * (1 / (10**solar_logOH))

    scale = (10**galaxy_logSiO) / (solar_SiO)

    return float("{:.2f}".format(scale))

# CONVERTING GALAXY SPREADSHEET INTO DATAFRAME

In [47]:
galaxy_vals = pd.read_excel('CLASSY_Metallicity_Te.xlsx')

df = galaxy_vals[[
    'Unnamed: 1', 'Unnamed: 3', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
    'Unnamed: 12'
]]

df1 = df.loc[6:].reset_index(drop=True)
df2 = df1.rename(
    columns={
        'Unnamed: 1': 'Galaxy',
        'Unnamed: 3': 'SDSS_density',
        'Unnamed: 7': 'SDSS_12logOH',
        'Unnamed: 8': 'OH_error (-)',
        'Unnamed: 9': 'OH_error (+)',
        'Unnamed: 12': 'MUSE_12logOH'
    })

galaxies = df2
galaxies.iloc[43, 2] = 7.77

galaxies.head(10)

,Galaxy,SDSS_density,SDSS_12logOH,OH_error (-),OH_error (+),MUSE_12logOH
0,J0021+0052,90,8.28,0.06,0.06,8.14
1,J0036-3333,-,-,-,-,7.47
2,J0127-0619,-,-,-,-,-
3,J0144+0453,-,-,-,-,-
4,J0337-0502,-,-,-,-,-
5,J0405-3648,-,-,-,-,7.22
6,J0808+3948,1250,-,-,-,-
7,J0823+2806,150,8.33,0.05,0.05,-
8,J0926+4427,100,8.05,0.08,0.06,-
9,J0934+5514,-,-,-,-,-


# DETERMINING METALLICITY SCALES FOR ALL GALAXIES

In [48]:
metallicities = []

for SDSS_logOH, MUSE_logOH in zip(galaxies['SDSS_12logOH'],
                                  galaxies['MUSE_12logOH']):
    if SDSS_logOH != '-':
        metal_val = find_metals(SDSS_logOH)
        metallicities.append(metal_val)
    elif MUSE_logOH != '-':
        metal_val = find_metals(MUSE_logOH)
        metallicities.append(metal_val)
    else:
        metallicities.append('-')

galaxies['CLOUDY metals and grains'] = metallicities

# we can now see our updated dataframe with the metallicity values
galaxies.head()

,Galaxy,SDSS_density,SDSS_12logOH,OH_error (-),OH_error (+),MUSE_12logOH,CLOUDY metals and grains
0,J0021+0052,90,8.28,0.06,0.06,8.14,0.39
1,J0036-3333,-,-,-,-,7.47,0.06
2,J0127-0619,-,-,-,-,-,-
3,J0144+0453,-,-,-,-,-,-
4,J0337-0502,-,-,-,-,-,-


# DETERMINING HYDROGEN LOG DENSITY FOR ALL GALAXIES

In [49]:
hdens = []

for density in galaxies['SDSS_density']:
    if (density != '-') and (density != '<100'):
        d_string = str(density)
        power = len(d_string) - 1
        hdens.append(power)
    elif density == '<100':
        hdens.append(1)
    else:
        hdens.append('-')

galaxies['log_density'] = hdens

galaxies.head(10)

,Galaxy,SDSS_density,SDSS_12logOH,OH_error (-),OH_error (+),MUSE_12logOH,CLOUDY metals and grains,log_density
0,J0021+0052,90,8.28,0.06,0.06,8.14,0.39,1
1,J0036-3333,-,-,-,-,7.47,0.06,-
2,J0127-0619,-,-,-,-,-,-,-
3,J0144+0453,-,-,-,-,-,-,-
4,J0337-0502,-,-,-,-,-,-,-
5,J0405-3648,-,-,-,-,7.22,0.03,-
6,J0808+3948,1250,-,-,-,-,-,3
7,J0823+2806,150,8.33,0.05,0.05,-,0.44,2
8,J0926+4427,100,8.05,0.08,0.06,-,0.23,2
9,J0934+5514,-,-,-,-,-,-,-


# SCALING THE ABUNDANCES OF N, C, AND Si

## Reading in Original Elemental Abundances

In [50]:
ele_df = pd.read_csv('metalliciy-N_O_SDSS_MUSE.txt', delimiter=r"\s+")

# add in zero column for Carbon to handle currently missing data
ele_df['CO_SDSS'] = 0.0
ele_df['CO_MUSE'] = 0.0

# add in zero column for Silicon to handle currently missing data
ele_df['Si_SDSS'] = 0.0
ele_df['Si_MUSE'] = 0.0

# add in error columns of placeholder error value 0.1
ele_df['N_err'] = 0.1
ele_df['C_err'] = 0.1
ele_df['Si_err'] = 0.1

# add in ionization column and error columns with placeholder value of -4 and 0.1, respectively
ele_df['Log U'] = -4
ele_df['Log U err'] = 0.1

ele_df.head()

,Galaxy,ne_SDSS,OH_SDSS,NO_SDSS,ne_MUSE,OH_MUSE,NO_MUSE,CO_SDSS,CO_MUSE,Si_SDSS,Si_MUSE,N_err,C_err,Si_err,Log U,Log U err
0,J0021+0052,90.0,8.28,-1.211556,120.0,8.14,-0.786478,0.0,0.0,0.0,0.0,0.1,0.1,0.1,-4,0.1
1,J0036-3333,0.0,0.00,0.000000,120.0,7.47,-1.557300,0.0,0.0,0.0,0.0,0.1,0.1,0.1,-4,0.1
2,J0127-0619,0.0,0.00,0.000000,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.1,0.1,0.1,-4,0.1
3,J0144+0453,0.0,0.00,0.000000,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.1,0.1,0.1,-4,0.1
4,J0337-0502,0.0,0.00,0.000000,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.1,0.1,0.1,-4,0.1


## Modifying J1044+0353 Parameters

In [51]:
ele_df.iloc[18, 14] = -1.77 # ionization
ele_df.iloc[18, 15] = 0.25  # ion error
ele_df.iloc[18, 1] = 200.0  # density
ele_df.iloc[18, 2] = 7.44   # OH log
ele_df.iloc[18, 3] = -1.35  # NO log
ele_df.iloc[18, 7] = -0.73  # CO log
ele_df.iloc[18, 11] = 0.06  # NO error
ele_df.iloc[18, 12] = 0.09  # CO error

In [52]:
ele_df.iloc[18]

Galaxy       J1044+0353
ne_SDSS             200
OH_SDSS            7.44
NO_SDSS           -1.35
ne_MUSE             160
OH_MUSE            7.49
NO_MUSE        -1.51771
CO_SDSS           -0.73
CO_MUSE               0
Si_SDSS               0
Si_MUSE               0
N_err              0.06
C_err              0.09
Si_err              0.1
Log U             -1.77
Log U err          0.25
Name: 18, dtype: object

## Creating Nitrogen Scale Column

In [53]:
NO_scale_list = []

for obj1, obj2 in zip(ele_df['NO_SDSS'], ele_df['NO_MUSE']):
    if int(obj1) != 0:
        NO_scale_list.append(N_scale(obj1))
    elif int(obj2) != 0:
        NO_scale_list.append(N_scale(obj2))
    else:
        NO_scale_list.append(1.00)

galaxies['NO_scale'] = NO_scale_list

## Creating Carbon Scale Column

In [54]:
CO_scale_list = []

for obj1, obj2 in zip(ele_df['CO_SDSS'], ele_df['CO_MUSE']):
    if int(obj1) != 0:
        CO_scale_list.append(C_scale(obj1))
    elif int(obj2) != 0:
        CO_scale_list.append(C_scale(obj2))
    else:
        CO_scale_list.append(1.00)

galaxies['CO_scale'] = CO_scale_list

## Creating Silicon Scale Column

In [55]:
SiO_scale_list = []

for obj1, obj2 in zip(ele_df['Si_SDSS'], ele_df['Si_MUSE']):
    if int(obj1) != 0:
        SiO_scale_list.append(Si_scale(obj1))
    elif int(obj2) != 0:
        SiO_scale_list.append(Si_scale(obj2))
    else:
        SiO_scale_list.append(1.00)

galaxies['SiO_scale'] = SiO_scale_list

## Creating an Ionization Column

In [56]:
galaxies['Log U'] = ele_df['Log U']

## Setting J1044+0052's Parameters

In [57]:
galaxies.iloc[18,1] = 200 # density
galaxies.iloc[18,7] = 2.0 # log density
galaxies.iloc[18,2] = 7.44
galaxies.iloc[18,3] = 0.1
galaxies.iloc[18,4] = 0.1
galaxies.iloc[18,6] = find_metals(7.44)
galaxies.iloc[18,8] = N_scale(-1.35)
galaxies.iloc[18,9] = C_scale(-0.73)

## Updated Galaxies Table With Scale Factors

In [58]:
galaxies.head(19)

,Galaxy,SDSS_density,SDSS_12logOH,OH_error (-),OH_error (+),MUSE_12logOH,CLOUDY metals and grains,log_density,NO_scale,CO_scale,SiO_scale,Log U
0,J0021+0052,90,8.28,0.06,0.06,8.14,0.39,1,0.45,1.00,1.0,-4.00
1,J0036-3333,-,-,-,-,7.47,0.06,-,0.20,1.00,1.0,-4.00
2,J0127-0619,-,-,-,-,-,-,-,1.00,1.00,1.0,-4.00
3,J0144+0453,-,-,-,-,-,-,-,1.00,1.00,1.0,-4.00
4,J0337-0502,-,-,-,-,-,-,-,1.00,1.00,1.0,-4.00
5,J0405-3648,-,-,-,-,7.22,0.03,-,0.21,1.00,1.0,-4.00
6,J0808+3948,1250,-,-,-,-,-,3,1.00,1.00,1.0,-4.00
7,J0823+2806,150,8.33,0.05,0.05,-,0.44,2,0.59,1.00,1.0,-4.00
8,J0926+4427,100,8.05,0.08,0.06,-,0.23,2,0.38,1.00,1.0,-4.00
9,J0934+5514,-,-,-,-,-,-,-,1.00,1.00,1.0,-4.00


# CREATING THE FINAL TABLE OF PARAMETERS

## Building the List of Galaxies and Their Indices

In [59]:
# creating a list of galaxy names
gal_grid_list = []

# creating a list of the indices
param_index = []

# number of original galaxies (45)
num_galaxies = len(galaxies)

# number of grids per parameter
n = 7

# empty lists of parameters to match indices
imet = []
iLu = []
idens = []
iNit = []
iCarb = []
iSic = []

# LOOP THROUGH EACH GALAXY:
for i in range(num_galaxies):

    # LOOP THROUGH METALLICITY:
    for x_1 in range(1, n + 1):

        # LOOP THROUGH LOG U:
        for x_2 in range(1, n + 1):

            # LOOP THROUGH DENSITY:
            for x_3 in range(1, n + 1):

                # LOOP THROUGH CNSi:
                for x_4 in range(1, n + 1):

                    ##### DEFINE THE METALLICITY SOLAR SCALE
                    if (galaxies.iloc[i, 2] != '-') and (galaxies.iloc[i, 4] !=
                                                         '-'):
                        # define the original OH value and error value
                        og_OHval = galaxies.iloc[i, 2]
                        err_OHval = galaxies.iloc[i, 4]

                        begin_logval = og_OHval - (3 * err_OHval)
                        final_val = find_metals(begin_logval + err_OHval *
                                                (x_1 - 1))
                        imet.append(final_val)
                    # if OH abundance ratio is null (make sure this doesn't mess up index)
                    else:
                        imet.append(1.0)

                    ##### DEFINE THE LOG_U PARAMETER STARRRR
                    if (galaxies.iloc[i,11]):
                        iLu.append(galaxies.iloc[i,11])
                    else:
                        iLu.append(-4)

                    ##### DEFINE THE DENSITY PARAMETER
                    if (galaxies.iloc[i,7] != '-'):
                        idens.append(galaxies.iloc[i,7])
                    else:
                        idens.append(2.0)

                    ##### DEFINE ELEMENTAL SCALES
                    ### NITROGEN
                    if (galaxies.iloc[i, 8] != 1.00):
                        # setting the error value
                        err1_NOval = ele_df.iloc[i, 11]

                        # if-else to set the original NO value
                        if (int(ele_df.iloc[i, 3]) != 0):
                            # NO SDSS
                            og1_NOval = ele_df.iloc[i, 3]
                        else:
                            # NO MUSE
                            og1_NOval = ele_df.iloc[i, 6]

                        begin1_NOval = og1_NOval - (3 * err1_NOval)
                        final1_NOval = N_scale(begin1_NOval + err1_NOval *
                                               (x_4 - 1))
                        iNit.append(final1_NOval)

                    # checking that C isn't null so that we can scale N with C
                    elif (galaxies.iloc[i, 9] != 1.00):
                        # setting the error value
                        err1_COval = ele_df.iloc[i, 12]

                        # if-else to set the original NO value
                        if (int(ele_df.iloc[i, 7]) != 0):
                            # CO SDSS
                            og1_COval = ele_df.iloc[i, 7]
                        else:
                            # CO MUSE
                            og1_COval = ele_df.iloc[i, 8]

                        begin1_COval = og1_COval - (3 * err1_COval)
                        final1_COval = C_scale(begin1_COval + err1_COval *
                                               (x_4 - 1))
                        iNit.append(final1_COval)

                    # appending zero if both Nitrogen and Carbon are null
                    else:
                        iNit.append(1.0)

                    ### CARBON
                    if (galaxies.iloc[i, 9] != 1.00):
                        # setting the error value
                        err2_COval = ele_df.iloc[i, 12]

                        # if-else to set the original NO value
                        if (int(ele_df.iloc[i, 7]) != 0):
                            # CO SDSS
                            og2_COval = ele_df.iloc[i, 7]
                        else:
                            # CO MUSE
                            og2_COval = ele_df.iloc[i, 8]

                        begin2_COval = og2_COval - (3 * err2_COval)
                        final2_COval = C_scale(begin2_COval + err2_COval *
                                               (x_4 - 1))
                        iCarb.append(final2_COval)

                    # if N is not null, scale C with N
                    elif (galaxies.iloc[i, 8] != 1.00):
                        # setting the error value
                        err2_NOval = ele_df.iloc[i, 11]

                        # if-else to set the original NO value
                        if (int(ele_df.iloc[i, 3]) != 0):
                            # NO SDSS
                            og2_NOval = ele_df.iloc[i, 3]
                        else:
                            # NO MUSE
                            og2_NOval = ele_df.iloc[i, 6]

                        begin2_NOval = og2_NOval - (3 * err2_NOval)
                        final2_NOval = N_scale(begin2_NOval + err2_NOval *
                                               (x_4 - 1))
                        iCarb.append(final2_NOval)

                    # append zero if both Carbon and Nitrogen are null
                    else:
                        iCarb.append(1.00)

                    ### SILICON
                    if (galaxies.iloc[i, 10] != 1.00):
                        # setting the error value
                        err_SiOval = ele_df.iloc[i, 13]

                        # if-else to set the original SiO value
                        if (int(ele_df.iloc[i, 9]) != 0):
                            # SiO SDSS
                            og_SiOval = ele_df.iloc[i, 9]
                        else:
                            # SiO MUSE
                            og_SiOval = ele_df.iloc[i, 10]

                        begin_SiOval = og_SiOval - (3 * err_SiOval)
                        final_SiOval = Si_scale(begin_SiOval + err_SiOval *
                                                (x_4 - 1))
                        iSic.append(final_SiOval)

                    # append one if Silicon is null
                    else:
                        iSic.append(1.00)

                    # define the galaxies column
                    gal_grid_list.append(galaxies.iloc[i, 0])

                    # define the index
                    param_index.append(
                        int(str(x_1) + str(x_2) + str(x_3) + str(x_4)))

# add all the lists to a table
param_df = pd.DataFrame()

param_df['Index'] = param_index
param_df['Galaxy'] = gal_grid_list
param_df['Metallicity'] = imet
param_df['LogU'] = iLu
param_df['H Dens'] = idens
param_df['N Scale'] = iNit
param_df['C Scale'] = iCarb
param_df['Si Scale'] = iSic

## Understanding Our Parameter DataFrame

In [60]:
param_df.head()

,Index,Galaxy,Metallicity,LogU,H Dens,N Scale,C Scale,Si Scale
0,1111,J0021+0052,0.26,-4.0,1.0,0.22,0.22,1.0
1,1112,J0021+0052,0.26,-4.0,1.0,0.28,0.28,1.0
2,1113,J0021+0052,0.26,-4.0,1.0,0.35,0.35,1.0
3,1114,J0021+0052,0.26,-4.0,1.0,0.45,0.45,1.0
4,1115,J0021+0052,0.26,-4.0,1.0,0.56,0.56,1.0


So given the way the loop is set up, we step through our grid initally from the last parameters, namely the elemental scales. We can see from the few values displayed above that this galaxy, J0021+0052, had a null value for either it's NO abundance or CO abundance but has the other value. Therefore, we scaled the missing value with the non-missing one and should expect that every iteration of this galaxy have equal scale factors for N and C.

In [61]:
print('Number of total galaxies (input files):', len(param_df))

Number of total galaxies (input files): 108045


In [62]:
print('Number of variations per galaxy (including original):', int(len(param_df) / 45))

Number of variations per galaxy (including original): 2401


## Checking that the Values Match the Indices

check by cross-referencing the values of parameters with the same single-value index but different across the general index

In [63]:
# defining our galaxy as a sub-dataframe
first_index = 0
last_index = 2401

temp_galaxy = param_df.iloc[first_index:last_index]

# creating a list of values for a specified parameter in a specified galaxy
vals = []

for i in range(len(temp_galaxy)):
    temp_index_list = [char for char in str(temp_galaxy.iloc[i, 0])]

    if (temp_index_list[3] == '1'):
        vals.append(temp_galaxy.iloc[i, 5])

# error counter as we iterate through our list of values
error = 0

for j in range(len(vals)):
    if vals[j] != vals[j - 1]:
        error += 1

print("The number of errrors in our elemental scale factor parameter:", error)

The number of errrors in our elemental scale factor parameter: 0


Could also try using data.loc 'where' method to access all iterations where the galaxy and the and the parameter variation are the same. Make this a list, then analyze the list to see if there are any differing values, which we can classify as errors. The expectation is that there either exist some n number of errors or zero errors. Given n, we can use this to deduce where the errors are being raised in the index, since we have a set of loops with differing ranges that are related to our value n.

## Creating a Dictionary to More Easily Access Individual Galaxies

In [64]:
# initialize an empty dictionary
pf_collection = {}

# iterate through original 45 galaxies
for gal in range(45):
    first_index = 2401 * gal
    last_index = 2401 * (gal + 1)
    temp_df = param_df.iloc[first_index:last_index]
    
    # reset the index key for each galaxy
    res_df = temp_df.reset_index(drop=True)
    
    # our key is the name of the galaxy
    key = str(galaxies.iloc[gal, 0])

    pf_collection[key] = res_df

# display our new dictionary
counter = 0

for key, value in pf_collection.items():
    if (counter < 5):
        print(key, ' : ', value)
        counter += 1

J0021+0052  :        Index      Galaxy  Metallicity  LogU  H Dens  N Scale  C Scale  Si Scale
0      1111  J0021+0052         0.26  -4.0     1.0     0.22     0.22       1.0
1      1112  J0021+0052         0.26  -4.0     1.0     0.28     0.28       1.0
2      1113  J0021+0052         0.26  -4.0     1.0     0.35     0.35       1.0
3      1114  J0021+0052         0.26  -4.0     1.0     0.45     0.45       1.0
4      1115  J0021+0052         0.26  -4.0     1.0     0.56     0.56       1.0
...     ...         ...          ...   ...     ...      ...      ...       ...
2396   7773  J0021+0052         0.59  -4.0     1.0     0.35     0.35       1.0
2397   7774  J0021+0052         0.59  -4.0     1.0     0.45     0.45       1.0
2398   7775  J0021+0052         0.59  -4.0     1.0     0.56     0.56       1.0
2399   7776  J0021+0052         0.59  -4.0     1.0     0.71     0.71       1.0
2400   7777  J0021+0052         0.59  -4.0     1.0     0.89     0.89       1.0

[2401 rows x 8 columns]
J0036-3333  

## A Guide to the Galaxy Index

In case the index is not clear, there are four numbers in the index with each number being associated with a different parameter that contains a grid: 

1st integer ----> <span class="mark">Metallicity</span>

2nd integer ----> <span class="mark">Nitrogen (N) scale factor</span>

3rd integer ----> <span class="mark">Carbon (C) scale factor</span>

4th integer ----> <span class="mark">Silicon (Si) scale factor</span>

Each integer ranges from 1-7 because we are taking the uncertainty of the previosuly found values for the original galaxies, and we are multiplying this uncertainty by 3 in both the (+) and (-) direction.

Therefore, a value of 6.4 with +/- uncertainty of 0.2 will give us a range of (5.8,7.0) with a step factor of 0.2. Thus, we will have galaxies with values for  [5.8, 6.0, 6.2, 6.4, 6.6, 6.8, 7.0] with the corresponding index  [1, 2, 3, 4, 5, 6, 7]

Given this, we can note that the original value for a galaxy will be marked with the index 4 for that parameter and the galaxy with all the original parameter values is that with index 4444.

## A Recap of All Existing Data Structures

Currently, we have multiple tables containing data in many different forms. We began with our galaxies dataframe, which we acquired from the excel sheet that was read in. This spreadsheet contained metallicity values and galaxy names. From here, we read in another excel sheet containing NO, CO, and SiO values that we could use to define our elemental scale factors (from solar). Using this, we created the ele_df dataframe to keep track of all existing parameters. Continuing further, we used both the galaxies dataframe and ele_df dataframe to create a final dataframe of all the galaxies with their parameters and the variations of these parameters, which serve to mimic the grid function in CLOUDY. Given that our table grew from length 45 (45 original galaxies) to 108045 (45 original gaalaxies with 2400 variations each), I decided to instanstiate a dictionary data structure, under the name pf_collection, so that we can access any individual original galaxy and its variations in a separate dataframe.

--- galaxies: table containing original 45 galaxies along with final parameters.

--- ele_df: table containing elemental (X)O values, as well as error values, that can be used to find the scale factors and their variations

--- param_df: final parameter table containing all parameters, all galaxies, and all variations (flattened)

--- pf_collection: dictionary containing param_df values, but broken up by the original galaxy (whihh serves as the key)

--- FITS file: different representation of pf_collection

# SAVING ALL THE DATA TO A FITS FILE

## Writing in Data to Cloud

In [65]:
# define file path
path_to_outfiles = '/Users/jat5764/Box/REU_2021_CLOUDY/JAYS_FILES/FITS'
outfile = path_to_outfiles + '/classy_cloudy_grid_info.fits'

# define the FITS header
hdr = pyfits.Header()
head = pyfits.PrimaryHDU(header=hdr)
hdu = pyfits.HDUList([head])

# add in the galaxy data from the dictionary into the FITS file
for i in range(1, len(galaxies) + 1):
    # define columns from parameter dictionary
    name = galaxies.iloc[i - 1, 0]

    GAL = pyfits.Column(name='GALAXY',
                        array=np.array(pf_collection[name]['Galaxy']),
                        format='A14',
                        unit='Galaxy Name')
    IND = pyfits.Column(name='INDEX',
                        array=np.array(pf_collection[name]['Index']),
                        format='J')
    MET = pyfits.Column(name='METALICITY',
                        array=np.array(pf_collection[name]['Metallicity']),
                        format='E',
                        unit='Percent Z_sol')
    LOGU = pyfits.Column(name='LOG U',
                         array=np.array(pf_collection[name]['LogU']),
                         format='E')
    HDENS = pyfits.Column(name='H Dens',
                          array=np.array(pf_collection[name]['H Dens']),
                          format='E',
                          unit='Log m/v')
    SCALE_N = pyfits.Column(name='SCALE N',
                            array=np.array(pf_collection[name]['N Scale']),
                            format='E',
                            unit='Percent Solar')
    SCALE_C = pyfits.Column(name='SCALE C',
                            array=np.array(pf_collection[name]['C Scale']),
                            format='E',
                            unit='Percent Solar')
    SCALE_SI = pyfits.Column(name='SCALE SI',
                             array=np.array(pf_collection[name]['Si Scale']),
                             format='E',
                             unit='Percent Solar')

    hdu.append(
        pyfits.BinTableHDU.from_columns(
            [GAL, IND, MET, LOGU, HDENS, SCALE_N, SCALE_C, SCALE_SI]))
    hdu[i].header['EXTNAME'] = name

# we set overwrite=True so that we can resave the files each time we update instead of making new ones
hdu.writeto(outfile, overwrite=True)
hdu.close()

As of now, this is clearly not working.

Questions and Notes:

--- Should we expect to have a large number of fits files or a singular file?

--- What should we expect to see for a given galaxy?

--- It might be worth displaying the data in python, effectively reading in our file and viewing it this way for easy reference and verification.

--- Should I go ahead on add in the units for the rest of the columns as well?

## Reading in FITS Data

In [66]:
with pyfits.open(
        '/Users/jat5764/Box/REU_2021_CLOUDY/JAYS_FILES/FITS/classy_cloudy_grid_info.fits'
) as hdu:
    fits_data = hdu[1].data
    fits_header = hdu[1].header

## Displaying FITS Data for Verification

In [67]:
# FITS header
print(fits_header)

XTENSION= 'BINTABLE'           / binary table extension                         BITPIX  =                    8 / array data type                                NAXIS   =                    2 / number of array dimensions                     NAXIS1  =                   42 / length of dimension 1                          NAXIS2  =                 2401 / length of dimension 2                          PCOUNT  =                    0 / number of group parameters                     GCOUNT  =                    1 / number of groups                               TFIELDS =                    8 / number of table fields                         TTYPE1  = 'GALAXY  '                                                            TFORM1  = '14A     '                                                            TUNIT1  = 'Galaxy Name'                                                         TTYPE2  = 'INDEX   '                                                            TFORM2  = 'J       '                    

In [68]:
# FITS data
print(fits_data)

[('J0021+0052', 1111, 0.26, -4., 1., 0.22, 0.22, 1.)
 ('J0021+0052', 1112, 0.26, -4., 1., 0.28, 0.28, 1.)
 ('J0021+0052', 1113, 0.26, -4., 1., 0.35, 0.35, 1.) ...
 ('J0021+0052', 7775, 0.59, -4., 1., 0.56, 0.56, 1.)
 ('J0021+0052', 7776, 0.59, -4., 1., 0.71, 0.71, 1.)
 ('J0021+0052', 7777, 0.59, -4., 1., 0.89, 0.89, 1.)]


# CREATING INPUT FILES FOR ALL GALAXY VARIATIONS

##  Defining a Function to Write Input Files

In [69]:
# here is an example input file that would be passed into CLOUDY (specific values are irrelevant)
def input_files(file, index, galaxy_txt, ion_param, hden, metal, n_scale,
                c_scale, si_scale):
    str_list = [
        '###############', '\ntitle', file + str(index), '\n###############',
        '\nset save prefix "' + file + str(index) + '"', '\ntable SED',
        '"' + galaxy_txt + '_stars.txt"', '\nionization parameter',
        str(ion_param), '\nhden',
        str(hden), '\nsphere', '\nabundances gass10 no grains',
        '\ngrains Orions', '\nmetals and grains',
        str(metal), '\nelement scale factor nitrogen',
        str(n_scale), '\nelement scale factor carbon',
        str(c_scale), '\nelement scale factor silicon',
        str(si_scale), '\niterate to convergence',
        '\nsave physical conditions ".cond"', '\nsave radius ".rad"',
        '\nsave overview ".ovr"', '\nsave ionization ".ion"',
        '\nsave grid ".out" last separate',
        '\nsave line list ".lin" "LineList_HII_DAB.dat" units Angstroms intrinsic absolute column last separate'
    ]

    return " ".join(str_list)


file = galaxies['Galaxy'][0]
str(
    input_files(file, pf_collection[galaxies['Galaxy'][0]]['Index'][0], file,
                pf_collection[galaxies['Galaxy'][0]]['LogU'][0],
                pf_collection[galaxies['Galaxy'][0]]['H Dens'][0],
                pf_collection[galaxies['Galaxy'][0]]['Metallicity'][0],
                pf_collection[galaxies['Galaxy'][0]]['N Scale'][0],
                pf_collection[galaxies['Galaxy'][0]]['C Scale'][0],
                pf_collection[galaxies['Galaxy'][0]]['Si Scale'][0]))

'############### \ntitle J0021+00521111 \n############### \nset save prefix "J0021+00521111" \ntable SED "J0021+0052_stars.txt" \nionization parameter -4.0 \nhden 1.0 \nsphere \nabundances gass10 no grains \ngrains Orions \nmetals and grains 0.26 \nelement scale factor nitrogen 0.22 \nelement scale factor carbon 0.22 \nelement scale factor silicon 1.0 \niterate to convergence \nsave physical conditions ".cond" \nsave radius ".rad" \nsave overview ".ovr" \nsave ionization ".ion" \nsave grid ".out" last separate \nsave line list ".lin" "LineList_HII_DAB.dat" units Angstroms intrinsic absolute column last separate'

The output presented immediately above is written in a way that can be easily read and translated into .txt file text using the .write() method below in 9.2 - 9.4. 

## Writing All Galaxy Variation Input Files For One CLASSY Galaxy

In [70]:
# example input filename and how it is generated
galaxies['Galaxy'][0] + '_' + str(
    pf_collection[galaxies['Galaxy'][0]]['Index'][23]) + '.in'

'J0021+0052_1143.in'

In [71]:
# looping through variations of galaxy J0021+0052
for i in range(len(pf_collection['J0021+0052'])):
    # file name and the path to access the file
    path = '/Users/jat5764/Box/REU_2021_CLOUDY/JAYS_FILES/CLOUDY_GALAXIES/GAL_J0021+0052/'
    file = galaxies['Galaxy'][0]
    variation = str(pf_collection[galaxies['Galaxy'][0]]['Index'][i])
    FILENAME = path + file + '_' + variation + '.in'

    # opening the file in order to write script
    ofile = open(FILENAME, "w")

    # function to write the script
    ofile.write(
        str(
            input_files(file + '_',
                        pf_collection[galaxies['Galaxy'][0]]['Index'][i],
                        file[:len(galaxies['Galaxy'][0]) - 5],
                        pf_collection[galaxies['Galaxy'][0]]['LogU'][i],
                        pf_collection[galaxies['Galaxy'][0]]['H Dens'][i],
                        pf_collection[galaxies['Galaxy'][0]]['Metallicity'][i],
                        pf_collection[galaxies['Galaxy'][0]]['N Scale'][i],
                        pf_collection[galaxies['Galaxy'][0]]['C Scale'][i],
                        pf_collection[galaxies['Galaxy'][0]]['Si Scale'][i])))

    # closing the file
    ofile.close()

## Writing Input Files for J1044+0053

In [72]:
# looping through variations of galaxy J0021+0052
for i in range(len(pf_collection['J1044+0353'])):
    # file name and the path to access the file
    path = '/Users/jat5764/Box/REU_2021_CLOUDY/JAYS_FILES/CLOUDY_GALAXIES/GAL_J1044+0353/'
    file = galaxies['Galaxy'][18]
    variation = str(pf_collection[galaxies['Galaxy'][18]]['Index'][i])
    FILENAME = path + file + '_' + variation + '.in'

    # opening the file in order to write script
    ofile = open(FILENAME, "w")

    # function to write the script
    ofile.write(
        str(
            input_files(
                file + '_', pf_collection[galaxies['Galaxy'][18]]['Index'][i],
                file[:len(galaxies['Galaxy'][0]) - 5] + '57',
                pf_collection[galaxies['Galaxy'][18]]['LogU'][i],
                pf_collection[galaxies['Galaxy'][18]]['H Dens'][i],
                pf_collection[galaxies['Galaxy'][18]]['Metallicity'][i],
                pf_collection[galaxies['Galaxy'][18]]['N Scale'][i],
                pf_collection[galaxies['Galaxy'][18]]['C Scale'][i],
                pf_collection[galaxies['Galaxy'][18]]['Si Scale'][i])))

    # closing the file
    ofile.close()

In [36]:
# looping through variations of galaxy J0021+0052
for i in range(4):
    # file name and the path to access the file
    path = '/Users/jat5764/Research/c17.02/source/'
    file = galaxies['Galaxy'][18]
    variation = str(pf_collection[galaxies['Galaxy'][18]]['Index'][i])
    FILENAME = path + file + '_' + variation + '.in'

    # opening the file in order to write script
    ofile = open(FILENAME, "w")

    # function to write the script
    ofile.write(
        str(
            input_files(
                file + '_', pf_collection[galaxies['Galaxy'][18]]['Index'][i],
                file[:len(galaxies['Galaxy'][0]) - 5] + '57',
                pf_collection[galaxies['Galaxy'][18]]['LogU'][i],
                pf_collection[galaxies['Galaxy'][18]]['H Dens'][i],
                pf_collection[galaxies['Galaxy'][18]]['Metallicity'][i],
                pf_collection[galaxies['Galaxy'][18]]['N Scale'][i],
                pf_collection[galaxies['Galaxy'][18]]['C Scale'][i],
                pf_collection[galaxies['Galaxy'][18]]['Si Scale'][i])))

    # closing the file
    ofile.close()

## Writing All Input Files For All 45 CLASSY Galaxies

#  PARSING THROUGH THE GRID FILES OF J1044+0353

## Defining Emission Lines

In [ ]:
_lines=['H  1 4861.4A',
        'H  1 6562.9A',        
        'He 2 1640.0A',
        'He 2 4686.0A',        
        'O  3 1661.0A',
        'O  3 1666.0A',
        'O II 3726.0A',
        'O II 3729.0A',
        'O  3 5007.0A',
        'O  3 4959.0A',
        'TOTL 4363.0A',        
        'C  3 1907.0A',
        'C  3 1910.0A',
        'C  4 1548.0A',
        'C  4 1551.0A',        
        'N  5 1239.0A',
        'N  4 1485.0A',
        'N  4 1486.0A',
        'N  2 6548.0A',
        'N  2 6584.0A',        
        'Si 4 1394.0A',
        'Si 4 1403.0A',
        'Si 3 1883.0A',
        'Si 3 1892.0A']

## Handling Line File (.lin)

In [ ]:
class Cloudy_Line():
    def __init__(self, filename):
        """
        a class to read in cloudy spectral line output file
        usage:
        file1=Cloudy_Line('grid000000000_.lin')
        then can look at:
        file1.data
        this uses the list of lines specified above
        """

        if not os.path.exists(filename):
            sys.stderr.write('Could not find file %s\n' % filename)
        else:
            self.filename = filename
            self.lines = _lines
            self.data = {}
            lines = open(filename).readlines()

            for line in lines:
                for specline in self.lines:
                    if specline in line:
                        self.data[specline] = float(line.split()[-3])
            for specline in self.lines:
                if not self.data.has_key(specline):
                    sys.stderr.write('Could not find line "%s" in file %s\n' %
                                     (specline, self.filename))
                    self.data[specline] = -999

## Handling Output Files (.out)

In [ ]:
class Cloudy_Output():
    def __init__(self, filename):
        self.logU = None
        self.Te = None
        self.elements = {}
        if not os.path.exists(filename):
            sys.stderr.write('Could not find file %s\n' % filename)
        else:
            self.filename = filename
            lines = open(self.filename).readlines()
            i = 0
            while i < len(lines):
                line = lines[i]
                if line.strip().startswith('IONIZE PARMET'):
                    self.logU = float(line.strip().split()[3])
                if line.strip().startswith('Radius'):
                    self.Te = float(line.strip().split()[1])
                if 'Log10 Mean Ionisation (over radius)' in line:
                    if not lines[i + 4].strip().startswith('Carbon'):
                        sys.stderr.write(
                            'Could not find Carbon data in file %s\n' %
                            filename)
                        print lines[i + 4]
                    if not lines[i + 5].strip().startswith('Nitrogen'):
                        sys.stderr.write(
                            'Could not find Nitrogen data in file %s\n' %
                            filename)
                    if not lines[i + 6].strip().startswith('Oxygen'):
                        sys.stderr.write(
                            'Could not find Oxygen data in file %s\n' %
                            filename)
                    self.elements['C'] = map(
                        float, lines[i + 4].replace('-', ' -').split()[1:])
                    self.elements['N'] = map(
                        float, lines[i + 5].replace('-', ' -').split()[1:])
                    self.elements['O'] = map(
                        float, lines[i + 6].replace('-', ' -').split()[1:])
                    self.elements['Si'] = map(
                        float, lines[i + 11].replace('-', ' -').split()[1:])

                i += 1

## Iterating Through the Produced Output Files

In [ ]:
# string defining the bases of all the produced files
base1 = 'grid00000000'
base2 = '_J1044+0353_grid'

# number of files
nruns = 2459
lineoutfile = 'cloudylines.dat'
utefile = 'logu_te.dat'
ionfile = 'ionfrac.dat'

# writing file data to separate file
file = open(lineoutfile, 'w')
ufile = open(utefile, 'w')
ifile = open(ionfile, 'w')

# iterating through all the produced files
for run in range(0, nruns):
    
    # handles the changing number of zeros in base1
    if (10 <= run < 100):
        ubase = base1[:-1]
    elif (100 <= run < 1000):
        ubase = base1[:-2]
    elif (urun > 1000):
        ubase = base1[:-3]
    # defining the .lin file
    linefile = ubase + str(run) + base2 + '.lin'
    # defining the .out file
    outfile = ubase + str(run) + base2 + '.out'
    
    str(run)
    str(linefile)
    str(outfile)

    file1 = Cloudy_Line(linefile)
    file2 = Cloudy_Output(outfile)

    # write headers
    if run == 0:
        file.write('#Run ')
        for linename in file1.lines:
            file.write(re.sub(' ', '_', linename) + ' ')
        file.write('\n')

        ufile.write('#Run LogU Te\n')

        ifile.write(
            '#Run CI CII CIII CIV NI NII NIII NIV OI OII OIII OIV SiI SiII SiIII SiIV\n'
        )

    # line data
    file.write(str(run) + ' ')
    for linename in file1.lines:
        file.write(str(file1.data[linename]) + ' ')
    file.write('\n')

    # Log U and Te data
    ufile.write('%i %3.2f %3.2e\n' % (run, file2.logU, file2.Te))

    # ionization fraction data
    ifile.write(
        '%i %4.3f %4.3f %4.3f %4.3f %4.3f %4.3f %4.3f %4.3f %4.3f %4.3f %4.3f %4.3f %4.3f %4.3f %4.3f %4.3f\n'
        % (run, file2.elements['C'][0], file2.elements['C'][1],
           file2.elements['C'][2], file2.elements['C'][3],
           file2.elements['N'][0], file2.elements['N'][1],
           file2.elements['N'][2], file2.elements['N'][3],
           file2.elements['O'][0], file2.elements['O'][1],
           file2.elements['O'][2], file2.elements['O'][3],
           file2.elements['Si'][0], file2.elements['Si'][1],
           file2.elements['Si'][2], file2.elements['Si'][3]))

file.close()
ufile.close()
ifile.close()